# `uasyncio` on CircuitPython

#### Requirements / Changes

* Lazy import isn't working!
* `utime` module (`ticks_XXX`)
* `core.py`: change `id` to `id_` 
    * urpc proxy objects have many-to-1 relationship with object on esp32 server
* core.SingletonGenerator: `__iter` -> `__await__`

#### TODO

* make urpc optional if sockets not used

In [1]:
%discover
%connect test-argon
%rsync
%softreset -q

test-argon                     serial:///dev/cu.usbmodem1414301        
test-stm32                     serial:///dev/cu.usbmodem208E307542522  
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
Connected to test-argon @ serial:///dev/cu.usbmodem1414301
UPDATE  lib/uasyncio/core.py


In [1]:
# start URPC client

# release UART
%softreset

from urpc import *

# check it out
import sys
sys_ = import_('sys')

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Platforms:
   host:    nRF52840
   server:  esp32


In [1]:
import uasyncio
from board import RGB_LED_RED, RGB_LED_GREEN, RGB_LED_BLUE
from digitalio import DigitalInOut
    
def make_led(pin):
    l = DigitalInOut(pin)
    l.switch_to_output()
    l.value = True
    return l

async def blink(led, period_ms):
    while True:
        led.value = False
        await uasyncio.sleep_ms(200)
        led.value = True
        await uasyncio.sleep_ms(max(10, period_ms-200))
        
async def main(led1, led2, led3):
    with make_led(led1) as l1, make_led(led2) as l2, make_led(led3) as l3:
        uasyncio.create_task(blink(l1, 300))
        uasyncio.create_task(blink(l2, 400))
        uasyncio.create_task(blink(l3, 500))
        await uasyncio.sleep_ms(3_000)
    print("main done!")

uasyncio.run(main(RGB_LED_RED, RGB_LED_GREEN, RGB_LED_BLUE))

main done!
